<a href="https://colab.research.google.com/github/Rohit1217/Vq_vae/blob/main/trainvqvae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import TensorDataset,DataLoader

from datasets_gen import get_mnist_
from vq_vae import VQ_VAE

In [ ]:
def get_device():
  return torch.device("cuda" if torch.cuda.is_available() else "cpu")
Device=get_device()

In [ ]:
data=get_mnist_()
traindata=TensorDataset(data,data)
trainloader=DataLoader(traindata,batch_size=128,shuffle=True)

In [ ]:
model=VQ_VAE()
model=model.to(Device)
epochs=100
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

In [ ]:
loss_list=[]
for epoch in range(epochs):
  total_loss=0
  count=0
  for input,target in trainloader:
    input,target=input.to(Device),target.to(Device)
    pred=model(input)
    loss=criterion(pred[0],pred[1],pred[2],target)

    total_loss+=loss.item()
    count+=1

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'total_loss={total_loss},avg_loss={total_loss/count},epoch={epoch}')